This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
!pip install pandas
!pip install lancedb
!pip install -U --quiet sentence-transformers
!pip install openpyxl
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 59.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 29.0 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 49.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.7/25.7 MB 49.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 33.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 14.8 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaul

In [1]:
import pandas as pd
import re

import os

os.environ["OPENAI_API_KEY"] = "voc-76835233612667733741246693c5e229dc21.94804767"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.llms import OpenAI
import openai

## Generate Real Estate Data

In [2]:
user_prompt = f"List at least 15 real estate properties."

In [3]:
response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
          {
            "role": "system",
            "content": """Your goal is to generate realistic real estate listings that includes neighborhood, price, 
            bedroom count, bathroom count, house size, description of the house and neighborhood description.
              
            Example: 
            Property1:
            Neighborhood: Green Oaks
            Price: $800,000
            Bedrooms: 3
            Bathrooms: 2
            House Size: 2,000 sqft
            Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
            Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
              """
          },
          {
            "role": "user",
            "content": user_prompt
          }
          ],
        temperature=1,
        #max_tokens=10000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )
# The response is a JSON object containing more information than the response. We want to return only the message content
#print(response)
home_list= response.choices[0].message.content

home_list

'Property 1:\nNeighborhood: Willow Creek\nPrice: $500,000\nBedrooms: 2\nBathrooms: 1\nHouse Size: 1,200 sqft\nDescription: Cozy 2-bedroom, 1-bathroom home in the desirable Willow Creek neighborhood. This charming house features a spacious living room, updated kitchen with stainless steel appliances, and a backyard perfect for entertaining. Enjoy the peaceful surroundings and close proximity to Willow Creek Park.\nNeighborhood Description: Willow Creek is known for its family-friendly atmosphere, tree-lined streets, and top-rated schools. Residents can enjoy outdoor activities at the nearby parks, as well as shopping and dining options in the quaint downtown area.\n\nProperty 2:\nNeighborhood: Oakridge Estates\nPrice: $750,000\nBedrooms: 4\nBathrooms: 3\nHouse Size: 2,800 sqft\nDescription: Stunning 4-bedroom, 3-bathroom home in the prestigious Oakridge Estates. This beautifully renovated house features a gourmet kitchen, luxurious master suite, and a spacious backyard with a pool. Perf

In [4]:
with open("./listings.txt", 'w') as file:
        file.write(home_list)

In [5]:
with open("./listings.txt", 'r') as file:
        homes_list =  file.read()

In [6]:
def parse_properties(text):
    properties = re.split(r'\n\nProperty \d+:\n', text.strip())
    properties = [prop for prop in properties if prop]  # Remove any empty strings

    property_list = []

    for property in properties:
        details = {}
        lines = property.split("\n")
        for line in lines:
            if line.startswith("Neighborhood: "):
                details["Neighborhood"] = line.split(": ", 1)[1]
            elif line.startswith("Price: "):
                details["Price"] = line.split(": ", 1)[1]
            elif line.startswith("Bedrooms: "):
                details["Bedrooms"] = line.split(": ", 1)[1]
            elif line.startswith("Bathrooms: "):
                details["Bathrooms"] = line.split(": ", 1)[1]
            elif line.startswith("House Size: "):
                details["House Size"] = line.split(": ", 1)[1]
            elif line.startswith("Description: "):
                details["Description"] = line.split(": ", 1)[1]
            elif line.startswith("Neighborhood Description: "):
                details["Neighborhood Description"] = line.split(": ", 1)[1]
        property_list.append(details)

    df = pd.DataFrame(property_list)

    df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)
    df['House Size'] = df['House Size'].replace('[\, sqft]', '', regex=True).astype(float)
    df['Bedrooms'] = pd.to_numeric(df['Bedrooms'], errors='coerce')
    df['Bathrooms'] = pd.to_numeric(df['Bathrooms'], errors='coerce')

    return df

homes_df = parse_properties(homes_list)
homes_df

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,Willow Creek,500000.0,2,1.0,1200.0,"Cozy 2-bedroom, 1-bathroom home in the desirab...",Willow Creek is known for its family-friendly ...
1,Oakridge Estates,750000.0,4,3.0,2800.0,"Stunning 4-bedroom, 3-bathroom home in the pre...",Oakridge Estates is known for its upscale home...
2,Pinecrest Heights,600000.0,3,2.5,2400.0,"Welcome to this lovely 3-bedroom, 2.5-bathroom...",Pinecrest Heights offers a peaceful mountain s...
3,River Oaks,1200000.0,5,4.0,3500.0,"Immaculate 5-bedroom, 4-bathroom home in the e...",River Oaks is known for its luxury real estate...
4,Sunset Hills,900000.0,4,3.5,3000.0,"Elegant 4-bedroom, 3.5-bathroom home in the so...",Sunset Hills offers a blend of luxury living a...
5,Lakeside Manor,650000.0,3,2.5,2200.0,"Charming 3-bedroom, 2.5-bathroom home in the s...",Lakeside Manor is a picturesque waterfront com...
6,Maple Grove,550000.0,3,2.0,1800.0,"Welcome to this charming 3-bedroom, 2-bathroom...","Maple Grove is known for its quiet streets, co..."
7,Mountain View Terrace,700000.0,4,3.0,2500.0,"Spacious 4-bedroom, 3-bathroom home with pictu...",Mountain View Terrace offers a peaceful escape...
8,Parkside Meadows,680000.0,3,2.5,2300.0,"Modern 3-bedroom, 2.5-bathroom home in the sce...",Parkside Meadows is a green community with acc...
9,Cedar Ridge Estates,850000.0,4,3.5,3200.0,"Luxurious 4-bedroom, 3.5-bathroom home in the ...",Cedar Ridge Estates is an upscale gated commun...


In [7]:
import openpyxl
homes_df.to_excel("./homes.xlsx", index=False)

In [8]:
df = pd.read_excel("./homes.xlsx")
df

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,Willow Creek,500000,2,1.0,1200,"Cozy 2-bedroom, 1-bathroom home in the desirab...",Willow Creek is known for its family-friendly ...
1,Oakridge Estates,750000,4,3.0,2800,"Stunning 4-bedroom, 3-bathroom home in the pre...",Oakridge Estates is known for its upscale home...
2,Pinecrest Heights,600000,3,2.5,2400,"Welcome to this lovely 3-bedroom, 2.5-bathroom...",Pinecrest Heights offers a peaceful mountain s...
3,River Oaks,1200000,5,4.0,3500,"Immaculate 5-bedroom, 4-bathroom home in the e...",River Oaks is known for its luxury real estate...
4,Sunset Hills,900000,4,3.5,3000,"Elegant 4-bedroom, 3.5-bathroom home in the so...",Sunset Hills offers a blend of luxury living a...
5,Lakeside Manor,650000,3,2.5,2200,"Charming 3-bedroom, 2.5-bathroom home in the s...",Lakeside Manor is a picturesque waterfront com...
6,Maple Grove,550000,3,2.0,1800,"Welcome to this charming 3-bedroom, 2-bathroom...","Maple Grove is known for its quiet streets, co..."
7,Mountain View Terrace,700000,4,3.0,2500,"Spacious 4-bedroom, 3-bathroom home with pictu...",Mountain View Terrace offers a peaceful escape...
8,Parkside Meadows,680000,3,2.5,2300,"Modern 3-bedroom, 2.5-bathroom home in the sce...",Parkside Meadows is a green community with acc...
9,Cedar Ridge Estates,850000,4,3.5,3200,"Luxurious 4-bedroom, 3.5-bathroom home in the ...",Cedar Ridge Estates is an upscale gated commun...


## Create Embeddings

In [9]:
import numpy as np
from sentence_transformers import SentenceTransformer
from typing import Union

MODEL_NAME = 'paraphrase-MiniLM-L6-v2'

def generate_embeddings(input_data: Union[str, list[str]]) -> np.ndarray:    
    model = SentenceTransformer(MODEL_NAME)
    embeddings = model.encode(input_data)
    return embeddings

/home/student/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [10]:
combined_texts = df['Description'] + " " + df['Neighborhood Description']
combined_embeddings = generate_embeddings(combined_texts.tolist())

df['Embedding'] = list(combined_embeddings)


#description_embeddings = generate_embeddings(df['Description'].tolist())
#neighborhood_embeddings = generate_embeddings(df['Neighborhood Description'].tolist())

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
combined_embeddings.shape

(15, 384)

## Store in Vector Database

In [12]:
import lancedb
from lancedb.pydantic import vector, LanceModel

class RealEstate(LanceModel):
    neighborhood: str
    price: float
    bedrooms:int
    bathrooms:int
    houseSize:float
    description: str
    neighDesc: str
    vector: vector(384) # size of embeddings
   # neighDesc: vector(384)
        

In [13]:
import pyarrow as pa

values = list(zip(
    df['Neighborhood'],
    df['Price'],
    df['Bedrooms'],
    df['Bathrooms'],
    df['House Size'],
    df['Description'],
    df['Neighborhood Description'],
    df['Embedding'],
))

keys = RealEstate.__annotations__.keys()
data = [dict(zip(keys, v)) for v in values]

db = lancedb.connect("~/.lancedb")
table_name = "real_estate"
db.drop_table(table_name, ignore_missing=True)
data = pa.Table.from_pylist(data, schema=RealEstate.to_arrow_schema())
table = db.create_table(table_name, data=data)

In [14]:
table.head().to_pandas()

,neighborhood,price,bedrooms,bathrooms,houseSize,description,neighDesc,vector
0,Willow Creek,500000.0,2,1,1200.0,"Cozy 2-bedroom, 1-bathroom home in the desirab...",Willow Creek is known for its family-friendly ...,"[0.17979175, -0.29748088, 0.36346433, 0.247457..."
1,Oakridge Estates,750000.0,4,3,2800.0,"Stunning 4-bedroom, 3-bathroom home in the pre...",Oakridge Estates is known for its upscale home...,"[0.5834801, -0.39588994, 0.11937329, 0.2529195..."
2,Pinecrest Heights,600000.0,3,2,2400.0,"Welcome to this lovely 3-bedroom, 2.5-bathroom...",Pinecrest Heights offers a peaceful mountain s...,"[0.21183795, -0.11090915, 0.060348827, 0.32521..."
3,River Oaks,1200000.0,5,4,3500.0,"Immaculate 5-bedroom, 4-bathroom home in the e...",River Oaks is known for its luxury real estate...,"[0.67375374, -0.22719987, -0.07884135, 0.13620..."
4,Sunset Hills,900000.0,4,3,3000.0,"Elegant 4-bedroom, 3.5-bathroom home in the so...",Sunset Hills offers a blend of luxury living a...,"[0.511041, 0.09430799, -0.03751888, 0.15344027..."


In [15]:
questions = [   
                "How big do you want your house to be?" ,
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?", 
                "What is your maximum price?" 
            ]
answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters.",
    "750000"
]
user_pref = " ".join(answers)

## User Preferences

In [16]:
import ipywidgets as widgets
from IPython.display import display, clear_output

text_boxes = [widgets.Textarea(placeholder=q, layout=widgets.Layout(width='100%')) for q in questions]

submit_button = widgets.Button(description="Submit")
output = widgets.Output()

user_pref = ""
max_price = float('inf')

def on_submit_clicked(b):
    global user_pref, max_price
    user_answers = [tb.value for tb in text_boxes[:-1]] #exclude last question
    user_pref = " ".join(user_answers)
    
    try:
        max_price = float(text_boxes[-1].value)
    except ValueError:
        max_price = float('inf')
    
    with output:
        clear_output()
        display(f"Combined User Preferences: {user_pref}")
        display(f"Maximum Price: {max_price}")

submit_button.on_click(on_submit_clicked)
display(*text_boxes, submit_button, output)

Textarea(value='', layout=Layout(width='100%'), placeholder='How big do you want your house to be?')

Textarea(value='', layout=Layout(width='100%'), placeholder='What are 3 most important things for you in choos…

Textarea(value='', layout=Layout(width='100%'), placeholder='Which amenities would you like?')

Textarea(value='', layout=Layout(width='100%'), placeholder='Which transportation options are important to you…

Textarea(value='', layout=Layout(width='100%'), placeholder='How urban do you want your neighborhood to be?')

Textarea(value='', layout=Layout(width='100%'), placeholder='What is your maximum price?')

Button(description='Submit', style=ButtonStyle())

Output()

In [26]:
user_pref, max_price

('I want very big house beacuse we are crowded family. luxury, fancy, extravagant. bathub, pool, shopping center. I will use my car and bus. I want to live in dynamic city center.',
 9000000.0)

In [27]:
query_embeds = generate_embeddings(user_pref)

## Search by Similarity

In [28]:
pd.set_option('display.max_colwidth', None)

def search_estate(embed, price=float('inf')):
    result = table.search(query_embeds).metric("cosine").where(f"price<={price}").limit(3).to_df()
    return result

result= search_estate(query_embeds,price=max_price)
result

/tmp/ipykernel_59/3314704255.py:4: UnsupportedWarning: to_df is unsupported as of 0.4.0. Use to_pandas() instead
  result = table.search(query_embeds).metric("cosine").where(f"price<={price}").limit(3).to_df()


,neighborhood,price,bedrooms,bathrooms,houseSize,description,neighDesc,vector,_distance
0,Oakridge Estates,750000.0,4,3,2800.0,"Stunning 4-bedroom, 3-bathroom home in the prestigious Oakridge Estates. This beautifully renovated house features a gourmet kitchen, luxurious master suite, and a spacious backyard with a pool. Perfect for those who love to entertain or simply relax in style.","Oakridge Estates is known for its upscale homes, large lots, and manicured landscaping. Residents can enjoy the privacy and serenity of the neighborhood while still being close to shopping centers, fine dining restaurants, and top-rated schools.","[0.5834801, -0.39588994, 0.11937329, 0.25291958, -0.2614146, -0.1681921, -0.29145432, -0.5201467, 0.12030552, 0.047041014, -0.26196066, -0.19209164, -0.11463349, 0.36400902, -0.26143846, 0.04541526, 0.23073936, 0.13753882, 0.15945232, 0.43020713, -0.21088076, -0.07264419, 0.073262826, -0.018082695, 0.14294064, 0.03994311, 0.042969353, 0.19699329, -0.25695714, -0.3808476, 0.20731187, -0.10118194, 0.07465602, -0.1676105, -0.3324507, 0.1403715, -0.2218089, -0.5088077, -0.042636853, -0.17275621, -0.5189221, 0.270187, 0.5430273, 0.16084945, -0.30532974, 0.494279, -0.07035437, -0.077872016, 0.30965322, -0.027157879, 0.07050074, -0.007663371, 0.2345454, 0.20676267, -0.38952145, 0.5800222, -0.08773792, -0.30221045, 0.021999095, 0.15487342, 0.49229234, 0.39095238, 0.17433561, 0.14734994, -0.44728318, 0.2936581, -0.18241866, 0.30172464, -0.26038697, -0.118413664, 0.074206255, -0.11762884, -0.17255041, -0.16770865, 0.16247013, 0.06675718, -0.043319847, -0.045865342, 0.02859807, 0.18493499, 0.078540586, -0.18082136, -0.36000976, 0.10753367, -0.30591226, -0.2817893, -0.108783856, -0.27843505, 0.0616911, -0.20471115, 0.5480257, 0.43087813, -0.35739356, -0.3464391, 0.14195317, -0.24404024, -0.039933763, -0.107541464, 0.19124478, 0.11355587, ...]",0.533749
1,Oakwood Estates,950000.0,5,4,3500.0,"Stunning 5-bedroom, 4.5-bathroom home in the prestigious Oakwood Estates. This custom-designed house features a grand foyer, formal living room, home theater, and a resort-style backyard with a pool and spa. Perfect for those who appreciate luxury living.","Oakwood Estates is an exclusive gated community known for its grand estates, lush landscaping, and private amenities. Residents can enjoy a luxurious lifestyle with access to golf courses, country clubs, and upscale shopping options in this upscale neighborhood.","[0.69241416, -0.094680294, 0.0957188, 0.20066704, -0.14628619, 0.021363698, -0.2984039, -0.47927296, 0.22154954, 0.23132436, -0.18307124, -0.1566832, -0.09524917, 0.4341472, -0.08012002, 0.21258397, -0.056597706, -0.074774705, 0.04786504, 0.4820935, -0.054702014, 0.016258879, 0.0314355, -0.084208935, 0.0006768755, -0.12944542, -0.13185945, 0.2009954, -0.24002515, -0.29108575, 0.29882446, -0.087806426, -0.028017756, 0.050850466, -0.30333173, 0.13328406, -0.2978954, -0.36975017, 0.023965128, -0.054266587, -0.39570513, 0.14790037, 0.4932756, 0.06958468, -0.21298021, 0.48687106, 0.19109723, -0.06998982, 0.23075701, 0.023752496, 0.03650022, 0.29308513, 0.101846606, 0.12095612, -0.2505637, 0.53989786, -0.13219883, -0.3102987, -0.23316953, 0.09489681, 0.46055484, 0.35340908, 0.054177716, 0.09691672, -0.75430804, 0.22658758, -0.006605226, 0.3422806, -0.04565078, -0.33964962, -0.10092672, -0.054459527, -0.40712613, -0.18906085, 0.24318072, 0.27090818, -0.20717406, -0.18023255, 0.12446414, 0.09484439, 0.039240777, 0.06998025, -0.23691474, -0.07911965, -0.1965612, -0.2566768, -0.005606722, -0.113372155, 0.027366998, -0.20878027, 0.66625375, 0.158656, -0.41371134, -0.24082154, 0.039685026, 0.13278891, 0.06377676, -0.18241586, 0.072824135, 0.083514065, ...]",0.534609
2,Maple Grove,550000.0,3,2,1800.0,"Welcome to this charming 3-bedroom, 2-bathroom home in the desirable Maple Grove neighborhood. This single-story house features a bright and open floor plan, vaulted ceilings, and a spacious backyard with a fruit tree orchard. I

In [30]:
result_str = result.iloc[:,:7].to_string(index=False)
print(result_str)

    neighborhood    price  bedrooms  bathrooms  houseSize                                                                                                                                                                                                                                                                                description                                                                                                                                                                                                                                                              neighDesc
Oakridge Estates 750000.0         4          3     2800.0                       Stunning 4-bedroom, 3-bathroom home in the prestigious Oakridge Estates. This beautifully renovated house features a gourmet kitchen, luxurious master suite, and a spacious backyard with a pool. Perfect for those who love to entertain or simply relax in style.                  Oakridge Estates is known for its upsc

## Personalized Listed Descriptions

In [31]:
user_prompt = f"The preferences of the customer is: {user_pref} and the maximum price customer \
willing to give is {max_price} \
The available real estate list is: {result_str}. Explain why these real estates are appropiate for the customer."

In [32]:
response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
          {
            "role": "system",
            "content": """Your goal is to generate descriptions to the real estates provided, by mentioning how neighborhood, price, 
            bedroom count, bathroom count, house size, description of the house and neighborhood description aligns with the customer's preferences.
            Do not alter factual information. Give the answer in same order as the provided list, explain each and all of the options. 
              """
          },
          {
            "role": "user",
            "content": user_prompt
          }
          ],
        temperature=1,
        #max_tokens=10000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
        )

recommend= response.choices[0].message.content

print(recommend)

1. The first option in Oakridge Estates offers a large house with 4 bedrooms and 3 bathrooms, totaling 2800.0 square feet. The description highlights a stunning and renovated home with a pool in the backyard, perfect for entertaining and relaxation. The neighborhood, Oakridge Estates, is known for its upscale homes, large lots, and beautiful landscaping, providing privacy and serenity. This option aligns with the customer's preference for a big house with luxury features like a pool and a focus on entertaining. Additionally, the neighborhood's proximity to shopping centers aligns with the customer's criteria.

2. The second option in Oakwood Estates is a luxurious 5-bedroom, 4.5-bathroom home spanning 3500.0 square feet, ideal for a crowded family. The house boasts custom design, a home theater, and a pool with a spa in the backyard. Oakwood Estates is described as an exclusive gated community with grand estates and lush landscaping, offering a luxurious lifestyle with access to upscal